# Obtener Movie Ratings usando Web Scraping

## Objetivos:

* Familiarizarse con la librería requests.
* Familiarizarse con la librería BeautifulSoup.
* Comprender los elementos HTML y el DOM.
* Analizar como extraer información usando BeautifulSoup.
* Extraer información del sitio web IMDb
* Entender como realizar un pequeño web-crawler.

## Usar la librería requests para descargar el contenido de la página


Para comenzar a hacer web scraping de una página web, primero necesitamos descargar la página utilizando la biblioteca de Python llamada ```requests```. La biblioteca requests realizará una solicitud GET a un servidor web, lo cual nos permitirá descargar el contenido HTML de la página web que deseamos. Existen varios tipos de solicitudes que podemos realizar utilizando requests, siendo GET solo uno de ellos.

#### Instalando la libreria ``requests``

Solo correr este código para instalación

In [ ]:
!pip install requests

In [ ]:
from requests import get

# request the server the content of the web page by using get()
# store the server’s response in the variable response.
response = get('http://www.imdb.com/search/title?release_date=2019&sort=num_votes,desc&page=1')

# print a small part of response's content by accessing its .text attribute
# (response is now a Response object).
print(response.text[:500])



<!DOCTYPE html>
<html
    xmlns:og="http://ogp.me/ns#"
    xmlns:fb="http://www.facebook.com/2008/fbml">
    <head>
         

        <meta charset="utf-8">




        <script type="text/javascript">var IMDbTimer={starttime: new Date().getTime(),pt:'java'};</script>

<script>
    if (typeof uet == 'function') {
      uet("bb", "LoadTitle", {wb: 1});
    }
</script>
  <script>(function(t){ (t.events = t.events || {})["csm_head_pre_title"] = new Date().getTime(); })(IMDbTimer);</script>
      


Como se puede observar en la primera línea de ```response.text```, el servidor nos ha enviado un documento HTML.

## Pasear el contenido HTML usando BeautifulSoup


Para analizar nuestro documento HTML y extraer información de él, utilizaremos un módulo de Python llamado BeautifulSoup. En la siguiente celda de código, realizaremos lo siguiente:

- Importar BeautifulSoup del paquete (package) bs4.
- Parsear ``response.text`` creando un BeautifulSoup object, y asignarlo al objecto a través de ``html_soup``.

#### Install ``BeautifulSoup`` library

Corre el siguiente código para instalar la librería:


In [ ]:
!pip install BeautifulSoup4

In [ ]:
from bs4 import BeautifulSoup

# The 'html.parser' argument indicates that we want to do the
# parsing using Python’s built-in HTML parser.
html_soup = BeautifulSoup(response.text, 'html.parser')

type(html_soup)

bs4.BeautifulSoup

## Understanding the HTML structure

Antes de emocionarte demasiado por el web scraping, es necesario que comprendas el HTML del sitio web del cual deseas hacer scraping. Ten en cuenta que cada sitio web tiene una estructura diferente.

 <img src="https://raw.githubusercontent.com/giturra/sicss-chile-2023/main/taller-web-scraping/inspector.png" width ="1000" height=1000 >

1. Click derecho en la página web.
2. Click izquierdo en ``Inspect``
3. Activa el botón del cursor de desplazamiento en la parte superior izquierda.

Cada película se encuentra en una etiqueta ```div``` con la clase ```lister-item-mode-advanced```. Utilicemos el método ```find_all()``` para extraer todos los contenedores ```div``` que tienen un atributo de clase ```lister-item mode-advanced```.

In [ ]:
movie_containers = html_soup.find_all('div', class_ = 'lister-item mode-advanced')
print(type(movie_containers))
print(len(movie_containers))

<class 'bs4.element.ResultSet'>
50


Como se muestra, hay 50 contenedores, lo que significa que hay 50 películas enumeradas en cada página.

<img src="https://raw.githubusercontent.com/giturra/sicss-chile-2023/main/taller-web-scraping/titulo.png" width ="700" height="500" >

Ahora seleccionaremos solo el primer contenedor y extraeremos, a su vez, cada elemento de interés:

    - The name of the movie.
    - The year of release.
    - The IMDB rating.
    - The Metascore.
    - Directors
    - The number of votes.
    - Gross
    
Comenzemos con ``first_movie``

In [ ]:
#stored the content of this container in the first_movie variable
first_movie = movie_containers[1]

In [ ]:
first_movie

<div class="lister-item mode-advanced">
<div class="lister-top-right">
<div class="ribbonize" data-caller="filmosearch" data-tconst="tt4154796"></div>
</div>
<div class="lister-item-image float-left">
<a href="/title/tt4154796/"> <img alt="復仇者聯盟：終局之戰" class="loadlate" data-tconst="tt4154796" height="98" loadlate="https://m.media-amazon.com/images/M/MV5BMTc5MDE2ODcwNV5BMl5BanBnXkFtZTgwMzI2NzQ2NzM@._V1_UX67_CR0,0,67,98_AL_.jpg" src="https://m.media-amazon.com/images/S/sash/4FyxwxECzL-U1J8.png" width="67"/>
</a> </div>
<div class="lister-item-content">
<h3 class="lister-item-header">
<span class="lister-item-index unbold text-primary">2.</span>
<a href="/title/tt4154796/">復仇者聯盟：終局之戰</a>
<span class="lister-item-year text-muted unbold">(2019)</span>
</h3>
<p class="text-muted">
<span class="certificate">PG-12</span>
<span class="ghost">|</span>
<span class="runtime">181 min</span>
<span class="ghost">|</span>
<span class="genre">
Action, Adventure, Drama            </span>
</p>
<div class=

## Scraping Data

A partir del html de la variable "first_movie" que hemos almacenado, vamos a utilizar "find" y "find_all" con "slicing de cadenas" para realizar la magia.

### El nombre del película

In [ ]:
first_movie.h3.a.text

'復仇者聯盟：終局之戰'

### Año de estreno

In [ ]:
first_movie.h3.find('span', class_ = 'lister-item-year text-muted unbold').text

'(2019)'

### El IMDB rating.

In [ ]:
first_movie.strong.text

'8.4'

### El Metascore.

In [ ]:
int(first_movie.find('span', class_ = 'metascore').text)

78

### Directores

Aquí complica más, ya que la clase contiene Directores y Estrellas. Por eso es recomendable utilizar slicing y splitting para extraer solo los directores. Puedes usar la misma lógica para extraer las estrellas también.

In [ ]:
first_movie.find('p', class_ = '')

<p class="">
    Directors:
<a href="/name/nm0751577/">Anthony Russo</a>, 
<a href="/name/nm0751648/">Joe Russo</a>
<span class="ghost">|</span> 
    Stars:
<a href="/name/nm0000375/">Robert Downey Jr.</a>, 
<a href="/name/nm0262635/">Chris Evans</a>, 
<a href="/name/nm0749263/">Mark Ruffalo</a>, 
<a href="/name/nm1165110/">Chris Hemsworth</a>
</p>

In [ ]:
# use slicer [2:-2] to select only directors' names
a = first_movie.find('p', class_ = '').text.split('Stars')[0].split('\n')[2:-2]

# join the string together
a = ''.join(a)

a

'Anthony Russo, Joe Russo'

### El número de votos

In [ ]:
int(first_movie.find_all('span', attrs = {'name':'nv'})[0]['data-value'])

1194708

### Gross

In [ ]:
first_movie.find_all('span', attrs = {'name':'nv'})[1]['data-value']

'858,373,000'

### Asegurarse que la data se encuentra en el idioma correspondiente, en este caso Inglés

Como nota adicional, si ejecutas el código desde un país donde el inglés no es el idioma principal, es muy probable que algunos de los nombres de las películas se traduzcan al idioma principal de ese país.

Esto ocurre porque el servidor infiere tu ubicación a partir de tu dirección IP. Incluso si te encuentras en un país donde el inglés es el idioma principal, es posible que aún obtengas contenido traducido. Esto puede suceder si estás utilizando una VPN mientras realizas las solicitudes GET.

Si te encuentras con este problema, pasa los siguientes valores al parámetro de encabezados (headers) de la función get():

In [ ]:
headers = {"Accept-Language": "en-US, en;q=0.5"}

Esto comunicará al servidor algo como "Deseo que el contenido lingüístico sea en inglés estadounidense (en-US). Si en-US no está disponible, también estaría bien otros tipos de inglés (en), aunque no tanto como en-US". El parámetro q indica el grado de preferencia por un determinado idioma. Si no se especifica, el valor se establece en 1 por defecto, como en el caso de en-US.

## Cambiando los parametros de la URL

Las URL siguen una lógica determinada a medida que las páginas web cambian. A medida que realizamos las solicitudes, solo tendremos que variar los valores de solo dos parámetros de la URL.

- ``release_date`` : Crea una lista llamada **years_url** y llénala con los valores correspondientes a los años 2000-2017.
- ``page`` : Crea una lista llamada **pages** y llénala con los valores correspondientes a las primeras 4 páginas.


In [ ]:
pages = [str(i) for i in range(1,5)]
years_url = [str(i) for i in range(2000,2024)]

In [ ]:
pages

['1', '2', '3', '4']

In [ ]:
years_url

['2000',
 '2001',
 '2002',
 '2003',
 '2004',
 '2005',
 '2006',
 '2007',
 '2008',
 '2009',
 '2010',
 '2011',
 '2012',
 '2013',
 '2014',
 '2015',
 '2016',
 '2017',
 '2018',
 '2019',
 '2020',
 '2021',
 '2022',
 '2023']

## Controlar el crawl-rate

¿Cuál es el motivo de hacerlo?

- Es menos probable que nuestro IP sea bloqueado al evitar sobrecargar el servidor con decenas de solicitudes por segundo.
- Evitar interrumpir y sobrecargar el servidor para que pueda responder a las solicitudes de otros usuarios.

Sería necesario contar con dos funciones:

1. ``sleep()``: Controla la velocidad del bucle. Pausará la ejecución del bucle durante una cantidad especificada de segundos.
2. ``randint()``: Para simular el comportamiento humano, variaremos el tiempo de espera entre las solicitudes. Genera de forma aleatoria enteros dentro de un intervalo especificado.

In [ ]:
from time import sleep
from random import randint

## Monitoreando el bucle mientras sigue en ejecución

El monitoreo es muy útil en el proceso de prueba y depuración, especialmente si vas a extraer información de cientos o miles de páginas web en una sola ejecución de código. A continuación, se muestran los parámetros que vamos a monitorear:

1. El **frequency (speed) of requests**: asegurarse que el programa no este sobrecargando el servidor.

    ``Frequency value = el número de requests / el tiempo transcurrido desde la primera solicitud.``

2. El **number of requests**: para detener el bucle en caso de que se exceda el número de solicitudes esperadas.

3. El **status code of our requests**: asegurarse de que el servidor esté enviando las respuestas adecuadas.

Vamos a experimentar con esta técnica de monitoreo a pequeña escala primero.

In [ ]:
from time import time

# Set a starting time using the time() function from the time module, and assign the value to start_time.
start_time = time()

# Assign 0 to the variable requests which we’ll use to count the number of requests.
request = 0

#Start a loop, and then with each iteration:
    #- Simulate a request.
    #- Increment the number of requests by 1.
    #- Pause the loop for a time interval between 8 and 15 seconds.
    #- Calculate the elapsed time since the first request, and assign the value to elapsed_time.
    #- Print the number of requests and the frequency.

for _ in range(5):
    request += 1
    sleep(randint(1,3))
    elapsed_time = time() - start_time
    print('Request: {}; Frequency: {} requests/s'.format(request, request/elapsed_time))

Request: 1; Frequency: 0.49938920340217274 requests/s
Request: 2; Frequency: 0.6655804558863936 requests/s
Request: 3; Frequency: 0.5991326205495593 requests/s
Request: 4; Frequency: 0.6657251586865262 requests/s
Request: 5; Frequency: 0.5548303422949268 requests/s


#### Clear_output()

Dado que vamos a realizar más de 5 solicitudes, nuestro trabajo se verá un poco desordenado a medida que se acumula la salida. Para evitar eso, borraremos la salida después de cada iteración y la reemplazaremos con información sobre la solicitud más reciente.

Como:

1. Usar la función ``clear_output()`` de ``IPython’s core.display module``.
2. Configurar ``wait parameter`` de la función clear_output() a ``True`` espera a reemplazar la salida actual hasta que aparezca alguna nueva salida.

In [ ]:
from IPython.core.display import clear_output

start_time = time()
request = 0

for _ in range(5):
    request += 1
    sleep(randint(1,3))
    current_time = time()
    elapsed_time = current_time - start_time
    print('Request: {}; Frequency: {} requests/s'.format(request, request/elapsed_time))
    clear_output(wait = True)

Request: 5; Frequency: 0.5542374686529675 requests/s


### Warnings

Para monitorear el código de estado, configuraremos el programa para que nos avise si algo no está bien. Una solicitud exitosa se indica con un código de estado 200. Utilizaremos la función "warn()" del módulo de advertencias para generar una advertencia si el código de estado no es 200.

Elegimos una advertencia en lugar de detener el bucle porque existe una buena posibilidad de que rasparemos suficientes datos, incluso si algunas de las solicitudes fallan. Solo romperemos el bucle si el número de solicitudes es mayor de lo esperado.

In [ ]:
from warnings import warn
warn("Warning Simulation")

<ipython-input-23-931ad44d99f9>:2: UserWarning: Warning Simulation
  warn("Warning Simulation")


## Ahora juntando todo

¡Uf! El trabajo duro está hecho, ahora vamos a juntar todo lo que hemos hecho hasta ahora.

* Importar las bibliotecas necesarias.
* Volver a declarar las variables de las listas para que vuelvan a estar vacías.
* Preparar el bucle.
* Recorrer la lista years_url en el intervalo de 2015 a 2023 y recorrer la lista pages en el intervalo de 1 a 2.
* Realizar las solicitudes GET dentro del bucle de las páginas.
* Dar al parámetro headers el valor correcto para asegurarnos de obtener solo contenido en inglés.
* Pausar el bucle por un intervalo de tiempo entre 8 y 15 segundos.
* Lanzar una advertencia para los códigos de estado no 200.
* Romper el bucle si el número de solicitudes es mayor al esperado.
* Convertir el contenido HTML de la respuesta en un objeto BeautifulSoup.
* Extraer todos los contenedores de películas de este objeto BeautifulSoup.
Recorrer todos estos contenedores.
* Extraer los datos si un contenedor tiene una puntuación Metascore.
* Extraer los datos si un contenedor tiene una recaudación, de lo contrario agregar "-".

In [ ]:
from requests import get
from bs4 import BeautifulSoup
from time import time, sleep
from random import randint
from IPython.core.display import clear_output
headers = {"Accept-Language": "en-US, en;q=0.5"}

# Redeclare the lists to store data in
names = []
years = []
imdb_ratings = []
metascores = []
directors = []
votes = []
gross = []

# Prepare the loop
start_time = time()
request = 0

pages = [str(i) for i in range(1,2)]
years_url = [str(i) for i in range(2015,2023)]

# For every year in the interval 2010-2019
for year_url in years_url:

    # For every page in the interval 1-4
    for page in pages:

        # Make a get request
        # Exp: https://www.imdb.com/search/title/?release_date=2019&sort=num_votes,desc&page=1
        response = get('http://www.imdb.com/search/title?release_date='+year_url+'&sort=num_votes,desc&page='+page, headers = headers)

        # Pause the loop
        sleep(randint(8,15))

        # Monitor the requests
        request += 1
        elapsed_time = time() - start_time
        print('Request:{}; Frequency: {} requests/s'.format(request, request/elapsed_time))
        clear_output(wait = True)

        # Throw a warning for non-200 status codes
        if response.status_code != 200:
            warn('Request: {}; Status code: {}'.format(request, response.status_code))

        # Break the loop if the number of requests is greater than expected
        # 4 pages * 10 years = 40 requests
        if request > 40:
            warn('Number of requests was greater than expected.')
            break

        # Parse the content of the request with BeautifulSoup
        page_html = BeautifulSoup(response.text, 'html.parser')

        # Select all the 50 movie containers from a single page
        mv_containers = page_html.find_all('div', class_ = 'lister-item mode-advanced')

        # For every movie of these 50
        for container in mv_containers:
            # If the movie has a Metascore, then:
            if container.find('div', class_ = 'ratings-metascore') is not None:

                # Scrape the name
                name = container.h3.a.text
                names.append(name)

                # Scrape the year
                year = container.h3.find('span', class_ = 'lister-item-year').text
                years.append(year)

                # Scrape the IMDB rating
                imdb = float(container.strong.text)
                imdb_ratings.append(imdb)

                # Scrape the Metascore
                m_score = container.find('span', class_ = 'metascore').text
                metascores.append(int(m_score))

                # Scrape the directors
                director = ''.join(container.find('p', class_ = '').text.split('Stars')[0].split('\n')[2:-2])
                directors.append(director)

                # Scrape the number of votes
                vote = container.find_all('span', attrs = {'name':'nv'})[0]['data-value']
                votes.append(int(vote))

                # If the movie has a Gross, then:
                if len(container.find_all('span', attrs = {'name':'nv'})) >= 2:

                    # Scrape the gross
                    gross_value = container.find_all('span', attrs = {'name':'nv'})[1]['data-value']
                    gross.append(gross_value)

                else:
                    gross.append("-")


Request:8; Frequency: 0.07932857577787808 requests/s


## Transformar la scrapeada data en un archivo CSV

En el siguiente bloque de código:

- Juntar los datos en un DataFrame de Pandas.
- Imprimir información sobre el DataFrame recién creado:
- Mostrar la últimas 10 entradas.

In [ ]:
import pandas as pd
movie_ratings = pd.DataFrame({'movie': names,
'year': years,
'imdb': imdb_ratings,
'metascore': metascores,
'directors': directors,
'votes': votes,
'gross($)': gross,
})
print(movie_ratings.info())
movie_ratings.tail(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 321 entries, 0 to 320
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   movie      321 non-null    object 
 1   year       321 non-null    object 
 2   imdb       321 non-null    float64
 3   metascore  321 non-null    int64  
 4   directors  321 non-null    object 
 5   votes      321 non-null    int64  
 6   gross($)   321 non-null    object 
dtypes: float64(1), int64(2), object(4)
memory usage: 17.7+ KB
None


,movie,year,imdb,metascore,directors,votes,gross($)
311,Triangle of Sadness,(2022),7.3,63,Ruben Östlund,145179,-
312,Puss in Boots: The Last Wish,(2022),7.9,73,"Joel Crawford, Januel Mercado",141446,"168,464,485"
313,Scream,(I) (2022),6.3,60,"Matt Bettinelli-Olpin, Tyler Gillett",140601,"81,641,405"
314,Morbius,(2022),5.2,35,Daniel Espinosa,139563,"73,865,530"
315,Turning Red,(2022),7.0,83,Domee Shi,137272,-
316,X,(II) (2022),6.6,79,Ti West,136679,-
317,The Lost City,(2022),6.1,60,"Aaron Nee, Adam Nee",136530,"105,344,029"
318,Smile,(V) (2022),6.5,68,Parker Finn,134361,-
319,Hustle,(2022),7.3,68,Jeremiah Zagar,133967,-
320,The Unbearable Weight of Massive Talent,(2022),7.0,68,Tom Gormican,129258,-


In [ ]:
movie_ratings.to_csv('movie_ratings_raw.csv')